In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

# Define the path to your JSON file
file_path = '/content/drive/My Drive/MasterThesis/flickr8k_dataset/combined_captions_data_flickr_final.json'

# Open and load the JSON data
with open(file_path, 'r') as file:
    captions_data = json.load(file)

In [ ]:
import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def summarize_captions(gpt2_caption, blip_caption, gpt2_weight=1, blip_weight=1):
    # Apply weights by concatenating the captions with their respective weights
    combined_text = (gpt2_caption + ' ') * gpt2_weight + (blip_caption + ' ') * blip_weight
    input_length = len(combined_text.split())  # Calculate the input length

    # Adjust summarization parameters based on the combined text length
    max_length_ratio = 0.75
    min_viable_length = 20
    max_length = max(min_viable_length, int(input_length * max_length_ratio))
    min_length = max(10, max_length // 2)

    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text

# Process each item in the JSON
for key, item in captions_data.items():
    gpt2_caption = item['generated_captions'].get('gpt2', '')
    blip_caption = item['generated_captions'].get('blip', '')

    # Set the weights for GPT-2 and BLIP captions
    gpt2_weight = 2  # Example: GPT-2 captions are given twice the importance of BLIP
    blip_weight = 1

    # Only proceed if both captions are available
    if gpt2_caption and blip_caption:
        summarized_caption = summarize_captions(gpt2_caption, blip_caption, gpt2_weight, blip_weight)

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity between the summarized caption and original captions
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        captions_data[key]['summarized_caption'] = summarized_caption
        captions_data[key]['most_similar_caption'] = most_similar_caption
        captions_data[key]['cosine_similarity_score'] = cosine_similarity_score

        # Print the information for verification
        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

# Specify the output path for the updated JSON data
output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_flickr.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Summarized Caption:  A little girl standing next to a wooden fence is a little girl that is standing on the steps outside of
Most Similar Original Caption: A girl going into a wooden building .
Cosine Similarity Score: 0.772780179977417
Summarized Caption:  a dog and a cat walking down a street in a series of pictures . There are two dogs that
Most Similar Original Caption: Two dogs on pavement moving toward each other .
Cosine Similarity Score: 0.8511483669281006
Summarized Caption:  a little boy sitting on a grass covered field  a little girl that is sitting on the ground in front of
Most Similar Original Caption: Young girl with pigtails painting outside in the grass .
Cosine Similarity Score: 0.6955264806747437
Summarized Caption:  There is a man laying on a bench with a dog next to him . There is also a man with
Most Similar Original Caption: A man lays on a bench while his dog sits by him .
Cosine Similarity Score: 0.703813374042511
Summarized Caption:  a man wearing a hat and gl

In [ ]:
import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def summarize_captions(gpt2_caption, blip_caption, gpt2_weight=1, blip_weight=1):
    # Apply weights by concatenating the captions with their respective weights
    combined_text = (gpt2_caption + ' ') * gpt2_weight + (blip_caption + ' ') * blip_weight
    input_length = len(combined_text.split())  # Calculate the input length

    # Adjust summarization parameters based on the combined text length
    max_length_ratio = 0.75
    min_viable_length = 20
    max_length = max(min_viable_length, int(input_length * max_length_ratio))
    min_length = max(10, max_length // 2)

    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text

# Process each item in the JSON
for key, item in captions_data.items():
    gpt2_caption = item['generated_captions'].get('gpt2', '')
    blip_caption = item['generated_captions'].get('blip', '')

    # Set the weights for GPT-2 and BLIP captions
    gpt2_weight = 1
    blip_weight = 2

    # Only proceed if both captions are available
    if gpt2_caption and blip_caption:
        summarized_caption = summarize_captions(gpt2_caption, blip_caption, gpt2_weight, blip_weight)

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity between the summarized caption and original captions
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        captions_data[key]['summarized_caption'] = summarized_caption
        captions_data[key]['most_similar_caption'] = most_similar_caption
        captions_data[key]['cosine_similarity_score'] = cosine_similarity_score

        # Print the information for verification
        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

# Specify the output path for the updated JSON data
output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_flickr.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


Summarized Caption:  A little girl standing next to a wooden fence is a little girl that is standing on the steps outside of a house . a little
Most Similar Original Caption: A girl going into a wooden building .
Cosine Similarity Score: 0.7807981967926025
Summarized Caption:  a dog and a cat walking down a street  there are two dogs that are playing with each other on the street
Most Similar Original Caption: Two dogs on pavement moving toward each other .
Cosine Similarity Score: 0.8622475862503052
Summarized Caption:  a little boy sitting on a grass covered field  there is a little girl that is sitting on the ground in front of a rainbow .
Most Similar Original Caption: A little girl is sitting in front of a large painted rainbow .
Cosine Similarity Score: 0.7697330117225647
Summarized Caption:  There is a man laying on a bench with a dog next to him . There is also a man sitting on a 
Most Similar Original Caption: A man lays on a bench while his dog sits by him .
Cosine Similarity

In [ ]:
import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def select_key_phrases(text, num_phrases=3):
    words = word_tokenize(text)
    # Placeholder for a more sophisticated key phrase selection logic
    return ' '.join(words[:num_phrases])

def create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight):
    # Determine primary and secondary based on weights
    if gpt2_weight >= blip_weight:
        primary_caption = gpt2_caption
        secondary_caption = blip_caption
    else:
        primary_caption = blip_caption
        secondary_caption = gpt2_caption

    # Extract sentences from the primary caption
    primary_sentences = sent_tokenize(primary_caption)
    # Extract key phrases from the secondary caption
    secondary_key_phrases = select_key_phrases(secondary_caption)

    # Combine sentences and key phrases for summarization input
    combined_text = ' '.join(primary_sentences) + ' ' + secondary_key_phrases
    return combined_text

def summarize_combined_text(combined_text):
    # Summarization parameters
    input_length = len(combined_text.split())
    max_length_ratio = 0.75
    min_viable_length = 20
    max_length = max(min_viable_length, int(input_length * max_length_ratio))
    min_length = max(10, max_length // 2)

    # Generate summary
    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text

# Process each item in the JSON
for key, item in captions_data.items():
    gpt2_caption = item['generated_captions'].get('gpt2', '')
    blip_caption = item['generated_captions'].get('blip', '')

    # Weights for GPT-2 and BLIP captions
    gpt2_weight = 0.53  # Example prior weight for GPT-2
    blip_weight = 0.47  # Example prior weight for BLIP

    if gpt2_caption and blip_caption:
        combined_text = create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight)
        summarized_caption = summarize_combined_text(combined_text)

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", padding=True, truncation=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        captions_data[key]['summarized_caption'] = summarized_caption
        captions_data[key]['most_similar_caption'] = most_similar_caption
        captions_data[key]['cosine_similarity_score'] = cosine_similarity_score

        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

# Specify the output path for the updated JSON data
output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_3_flickr.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Summarized Caption:  A little girl standing next to a wooden fence there is a little girl . a little
Most Similar Original Caption: A girl going into a wooden building .
Cosine Similarity Score: 0.762624979019165


Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Summarized Caption:  a dog and a cat walking down a street there are two dogs and cats walking down
Most Similar Original Caption: Two dogs on pavement moving toward each other .
Cosine Similarity Score: 0.8539887070655823


Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


...

Summarized Caption:  There is a little boy standing in the water with a surfboard there is a surf
Most Similar Original Caption: A young boy wades in the ocean .
Cosine Similarity Score: 0.7608844041824341


Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Summarized Caption:  a boy is sitting on a boat in the water there is a boy sitting in the
Most Similar Original Caption: "A child holds on to the side of a small , weathered , green watercraft on the open sea ."
Cosine Similarity Score: 0.7129660844802856


Your max_length is set to 20, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Summarized Caption:  A young boy running across a grassy field there is a picture of a young boy
Most Similar Original Caption: A small boy is playing in an open field .
Cosine Similarity Score: 0.8036000728607178
Summarized Caption:  A sheep and a dog are in a field there is a field where there is no
Most Similar Original Caption: A fluffy white dog and two white sheep smell each other .
Cosine Similarity Score: 0.7725586295127869
JSON file has been updated and saved.


In [ ]:
import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def select_key_phrases(text, num_phrases=3):
    words = word_tokenize(text)
    # Placeholder for a more sophisticated key phrase selection logic
    return ' '.join(words[:num_phrases])

def create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight):
    # Determine primary and secondary based on weights
    if gpt2_weight >= blip_weight:
        primary_caption = gpt2_caption
        secondary_caption = blip_caption
    else:
        primary_caption = blip_caption
        secondary_caption = gpt2_caption

    # Extract sentences from the primary caption
    primary_sentences = sent_tokenize(primary_caption)
    # Extract key phrases from the secondary caption
    secondary_key_phrases = select_key_phrases(secondary_caption)

    # Combine sentences and key phrases for summarization input
    combined_text = ' '.join(primary_sentences) + ' ' + secondary_key_phrases
    return combined_text

def summarize_combined_text(combined_text):
    # Summarization parameters
    input_length = len(combined_text.split())
    max_length_ratio = 0.75
    min_viable_length = 20
    max_length = max(min_viable_length, int(input_length * max_length_ratio))
    min_length = max(10, max_length // 2)

    # Generate summary
    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text


# Process each item in the JSON
for key, item in captions_data.items():
    gpt2_caption = item['generated_captions'].get('gpt2', '')
    blip_caption = item['generated_captions'].get('blip', '')

    # Weights for GPT-2 and BLIP captions
    gpt2_weight = 0.47  # Example prior weight for GPT-2
    blip_weight = 0.53  # Example prior weight for BLIP

    if gpt2_caption and blip_caption:
        combined_text = create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight)
        summarized_caption = summarize_combined_text(combined_text)

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", padding=True, truncation=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        captions_data[key]['summarized_caption'] = summarized_caption
        captions_data[key]['most_similar_caption'] = most_similar_caption
        captions_data[key]['cosine_similarity_score'] = cosine_similarity_score

        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

# Specify the output path for the updated JSON data
output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_3_flickr.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Your max_length is set to 20, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Summarized Caption:  there is a little girl that is standing on the steps outside of a house a little
Most Similar Original Caption: A little girl climbing the stairs to her playhouse .
Cosine Similarity Score: 0.8079636096954346
Summarized Caption:  There are two dogs that are playing with each other on the street a dog and a
Most Similar Original Caption: A black dog and a spotted dog are fighting
Cosine Similarity Score: 0.8483843207359314


Your max_length is set to 20, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Summarized Caption:  There is a little girl that is sitting on the ground in front of a rainbow a
Most Similar Original Caption: There is a girl with pigtails sitting in front of a rainbow painting .
Cosine Similarity Score: 0.8103453516960144
Summarized Caption:  A man laying on a bench with a dog next to him a man laying there is
Most Similar Original Caption: A man lays on a bench while his dog sits by him .
Cosine Similarity Score: 0.9093778133392334


Your max_length is set to 20, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


...

Summarized Caption:  There is a young boy riding a skateboard down the street a little boy . there
Most Similar Original Caption: A young boy runs aross the street .
Cosine Similarity Score: 0.8380597829818726
Summarized Caption:  There is a dog that is jumping over a log in the grass a dog standing .
Most Similar Original Caption: The black dog jumped the tree stump .
Cosine Similarity Score: 0.7981563806533813


Your max_length is set to 20, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


Summarized Caption:  A dog that is running in the snow with a frisbee in it's mouth
Most Similar Original Caption: A dog running through snow .
Cosine Similarity Score: 0.7913511395454407


Your max_length is set to 20, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


Summarized Caption:  Two people standing in the snow with skis on two people standing standing . there are
Most Similar Original Caption: A man skis past another man displaying paintings in the snow .
Cosine Similarity Score: 0.7498085498809814
